In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
import pickle as pkl

In [2]:
data_path = 'C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/dummy_data/processed_data/preprocessed_data_submission_competitor_group_test.pkl'

with open(data_path, 'rb') as f:
    data_dict = pkl.load(f)

print(data_dict.keys())

dict_keys(['input_data', 'output_data', 'data'])


# Data processing

In [3]:
print(data_dict["input_data"]["vehicle_id"].shape)

(9, 1)


In [4]:
# train ds
num_price_control_features = data_dict["input_data"]["price_controls"].shape[1]
num_vehicle_control_features = data_dict["input_data"]["vehicle_controls"].shape[1]
num_macro_control_features = data_dict["input_data"]["macro_controls"].shape[1]

inputs_dict = {
    "time_idx": data_dict["input_data"]["time_idx"]/data_dict["input_data"]["time_idx"].max() + 1,
    "quarter_sin": data_dict["input_data"]["quarter_sin"]-1,
    "quarter_cos": data_dict["input_data"]["quarter_cos"]-1,
    "country_id": data_dict["input_data"]["country_id"],
    "brand_id": data_dict["input_data"]["brand_id"],
    "category_id": data_dict["input_data"]["category_id"],
    "price_controls": data_dict["input_data"]["price_controls"],
    "vehicle_mix": data_dict["input_data"]["vehicle_controls"],
    "total_spend": data_dict["input_data"]["total_spend"],   
    "macro_controls": data_dict["input_data"]["macro_controls"],
}

targets = (data_dict["input_data"]["y_true"]/100, data_dict["input_data"]["group_id"])

BATCH_SIZE = 30
train_ds = tf.data.Dataset.from_tensor_slices((inputs_dict, targets))
# train_ds = train_ds.shuffle(buffer_size=1024).batch(BATCH_SIZE)
train_ds = train_ds.batch(BATCH_SIZE)

In [5]:
# # predict_x
# predict_x = {
#     "time_idx": data_dict["output_data"]["time_idx"]/data_dict["output_data"]["time_idx"].max() + 1,
#     "quarter_sin": data_dict["output_data"]["quarter_sin"] - 1,
#     "quarter_cos": data_dict["output_data"]["quarter_cos"] - 1,
#     "country_id": data_dict["output_data"]["country_id"],
#     "brand_id": data_dict["output_data"]["brand_id"],
#     "category_id": data_dict["output_data"]["category_id"],
#     "price_controls": data_dict["output_data"]["price_controls"],
#     "vehicle_mix": data_dict["output_data"]["vehicle_controls"],
#     "total_spend": data_dict["output_data"]["total_spend"],
#     "macro_controls": data_dict["output_data"]["macro_controls"],
# }

# test_targets = (data_dict["output_data"]["y_true"]/100, data_dict["output_data"]["group_id"])

In [6]:
test_path = 'C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/dummy_data/processed_data/preprocessed_data_submission_format_competitor_group.pkl'
with open(test_path, 'rb') as f:
    test_dict = pkl.load(f)

predict_x = {
    "time_idx": test_dict["time_idx"]/test_dict["time_idx"].max(),
    "quarter_sin": test_dict["quarter_sin"]-1,
    "quarter_cos": test_dict["quarter_cos"]-1,
    "country_id": test_dict["country_id"],
    "brand_id": test_dict["brand_id"],
    "category_id": test_dict["category_id"],
    "price_controls": test_dict["price_controls"],
    "vehicle_mix": test_dict["vehicle_controls"],
    "total_spend": test_dict["total_spend"],
    "macro_controls": test_dict["macro_controls"],
}

# Model Arch

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# -----------------------
# Hyperparams
# -----------------------
COUNTRY_VOCAB = 3
BRAND_VOCAB = 50
CATEGORY_VOCAB = 6
COUNTRY_EMB = 1
BRAND_EMB = 1
CATEGORY_EMB = 1
TIME_HIDDEN = 16
CAT_HIDDEN = 128
CONTROL_HIDDEN = 64
DROPOUT = 0.5
VEHICLE_COUNT = 9
VEHICLE_EMB = 128


# -----------------------
# Inputs
# -----------------------
time_idx_in = layers.Input(shape=(1,), dtype="float32", name="time_idx")
quarter_sin_in = layers.Input(shape=(1,), dtype="float32", name="quarter_sin")
quarter_cos_in = layers.Input(shape=(1,), dtype="float32", name="quarter_cos")

country_in = layers.Input(shape=(1,), dtype="int32", name="country_id")
brand_in = layers.Input(shape=(1,), dtype="int32", name="brand_id")
category_in = layers.Input(shape=(1,), dtype="int32", name="category_id")

# vehicle id not required if we pass mix; keep only if you have per-row vehicle id
vehicle_id_in = layers.Input(shape=(1,), dtype="int32", name="vehicle_id")  # optional

# controls split by logical groups
price_controls_in = layers.Input(shape=(num_price_control_features,), dtype="float32", name="price_controls")
macro_controls_in = layers.Input(shape=(num_macro_control_features,), dtype="float32", name="macro_controls")

# total spend and vehicle mix separate
total_spend_in = layers.Input(shape=(1,), dtype="float32", name="total_spend")
vehicle_mix_in = layers.Input(shape=(VEHICLE_COUNT,), dtype="float32", name="vehicle_mix")  # fractions sum to 1

# -----------------------
# Time subnet
# -----------------------
# ---------------- TIME BLOCK ---------------- #

# --- Cast time index to float ---
time_float = layers.Lambda(lambda x: tf.cast(x, tf.float32))(time_idx_in)  # (batch,1)

# --- Global linear trend ---
global_trend = layers.Dense(1, use_bias=False, activation=None, name="global_trend")(time_float)

# --- Brand-specific slope * time ---
brand_slope = layers.Embedding(BRAND_VOCAB, 1, name="brand_slope")(brand_in)  # (batch,1,1)
brand_slope = layers.Reshape((1,))(brand_slope)                                 # (batch,1)
brand_slope = layers.Activation("tanh")(brand_slope)                            # now in [0, 1]
brand_slope = layers.Lambda(lambda x: 10 * x)(brand_slope)                       # now in [-0.5, 0.5]
brand_trend = layers.Multiply(name="brand_trend")([brand_slope, time_float])    # (batch,1)
brand_trend = layers.Dense(1, use_bias=False, activation=None, name="brand_trend_scale")(brand_trend)

# --- Country-specific slope * time ---
country_slope = layers.Embedding(COUNTRY_VOCAB, 1, name="country_slope")(country_in)
country_slope = layers.Reshape((1,))(country_slope)
country_slope = layers.Activation("tanh")(country_slope)                            # now in [0, 1]
# country_slope = layers.Lambda(lambda x: x - 0.5)(country_slope)                       # now in [-0.5, 0.5]
country_trend = layers.Multiply(name="country_trend")([country_slope, time_float])
country_trend = layers.Dense(1, use_bias=False, activation=None, name="country_trend_scale")(country_trend)

# --- Country-specific seasonality (sin/cos) ---
season_vec = layers.Concatenate()([quarter_sin_in, quarter_cos_in])  # (batch,2)
country_season = layers.Embedding(COUNTRY_VOCAB, 2, embeddings_regularizer=tf.keras.regularizers.l2(1e-3), name="country_season")(country_in)  # (batch,1,2)
country_season = layers.Reshape((2,))(country_season)                                      # (batch,2)
country_season = layers.Activation("tanh")(country_season)                            # now in [-1, 1]
# country_season = layers.Lambda(lambda x: x*0.5)(country_season)                       # now in [-0.5, 0.5]

season_out = layers.Dot(axes=1, name="season_out")([country_season, season_vec])           # (batch,1)

# --- Final time_out (add everything) ---
time_out = layers.Add(name="time_out")([brand_trend, season_out])  # (batch,1)

# -----------------------
# Country subnet
# -----------------------
c_emb = layers.Embedding(input_dim=COUNTRY_VOCAB, output_dim=COUNTRY_EMB, name="country_emb")(country_in)
c = layers.Flatten()(c_emb)
c = layers.Dense(CAT_HIDDEN, activation="relu")(c)
country_out = layers.Dense(1, activation=None, name="country_out")(c)
# country_out = layers.Activation("sigmoid")(country_out)

# -----------------------
# Brand subnet
# -----------------------
b_emb = layers.Embedding(input_dim=BRAND_VOCAB, output_dim=BRAND_EMB, name="brand_emb")(brand_in)
b = layers.Flatten()(b_emb)
b = layers.Dense(CAT_HIDDEN, activation="relu")(b)
brand_out = layers.Dense(1, activation=None, name="brand_out")(b)
# brand_out = layers.Activation("sigmoid")(brand_out)

# -----------------------
# Category subnet (country + brand)
# -----------------------
c_emb = layers.Embedding(input_dim=CATEGORY_VOCAB, output_dim=CATEGORY_EMB, name="category_emb")(category_in)
c = layers.Flatten()(c_emb)
cat_hidden = layers.Dense(CAT_HIDDEN, activation="relu", name="cat_hidden")(c)
category_out = layers.Dense(1, activation=None, name="category_out")(cat_hidden)
# category_out = layers.Activation("sigmoid")(category_out)

# -----------------------
# Controls subnets (price / macro controls) -> scalar each
# -----------------------
# Price controls
p = layers.Dense(CONTROL_HIDDEN, activation="relu")(price_controls_in)
p = layers.Dropout(DROPOUT)(p)
p = layers.Dense(CONTROL_HIDDEN // 2, activation="relu")(p)
p_out = layers.Dense(1, activation=None, name="price_ctrl_out")(p)  
# p_out = layers.Activation("sigmoid")(p_out)

# Macro controls
m = layers.Dense(CONTROL_HIDDEN, activation="relu")(macro_controls_in)
m = layers.Dropout(DROPOUT)(m)
m = layers.Dense(CONTROL_HIDDEN // 2, activation="relu")(m)
m_out = layers.Dense(1, activation=None, name="macro_ctrl_out")(m)
# m_out = layers.Activation("sigmoid")(m_out)

# -----------------------
# Vehicle mix -> learned effect
# (simple approach: learn mapping from mix vector to an effect scalar)
# -----------------------
vm = layers.Dense(VEHICLE_EMB, activation="relu", name="vehicle_mix_emb")(vehicle_mix_in)  # (batch, emb)
vehicle_effect = layers.Dense(1, activation=None, name="vehicle_effect")(vm)              # (batch,1)

# combine total_spend and vehicle_effect -> per-row vehicle contribution
vehicle_spend_out = layers.Multiply(name="vehicle_spend_out")([total_spend_in, vehicle_effect])  # (batch,1)

# combine p and vehicle controls
business_ctrl_concat = layers.Concatenate(name="business_ctrl_concat")([p_out, vehicle_spend_out])  # (batch,2)
business_ctrl_hidden = layers.Dense(8, activation=None, name="business_ctrl_hidden")(business_ctrl_concat)  # (batch,1)
business_ctrl_out = layers.Dense(1, activation=None, name="business_ctrl_out")(business_ctrl_hidden)  # (batch,1)
# business_ctrl_out = layers.Activation("sigmoid")(business_ctrl_out)

# -----------------------
# Baseline and variability split (explicit 90/10)
# -----------------------
baseline = layers.Add(name="baseline")([time_out, brand_out, country_out, m_out, category_out])  # (batch,1)
variability = layers.Lambda(lambda x: 1.0 * x, name="variability")(business_ctrl_out)  # enforce ~10% effect scale



# final additive logit
add_logit = layers.Add(name="add_logit")([baseline, variability])  # (batch,1)
# add_logit = layers.Dense(1, activation=None, name="add_logit")(layers.Concatenate()([baseline, variability]))  # (batch,1)

# -----------------------
# Build model
# -----------------------
model = keras.Model(
    inputs=[
        time_idx_in, quarter_sin_in, quarter_cos_in,
        country_in, brand_in, category_in,
        price_controls_in, vehicle_mix_in, macro_controls_in,
        total_spend_in
    ],
    outputs=[add_logit, variability, global_trend, brand_trend, country_trend, season_out, time_out, baseline, variability],
    name="gam_vehicle_mix_model"
)

model.summary()

Model: "gam_vehicle_mix_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ brand_id            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ price_controls      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ brand_slope         │ (None, 1, 1)      │         50 │ brand_id[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │        256 │ price_controls[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vehicle_mix         │ (None, 9)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 1)         │          0 │ brand_slope[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ country_id          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vehicle_mix_emb     │ (None, 128)       │      1,280 │ vehicle_mix[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 1)         │          0 │ reshape_3[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_idx            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ country_season      │ (None, 1, 2)      │          6 │ country_id[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ macro_controls      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_id         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ total_spend         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vehicle_effect      │ (None, 1)         │        129 │ vehicle_mix_emb[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 1)         │          0 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 7,456 (29.12 KB)

 Trainable params: 7,456 (29.12 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
def fractions_to_logits_tf(y_true, group_ids, eps=1e-8):
    """
    Convert true fractions (per group) to centered logits.
    Args:
        y_true:    (N,) float32 tensor, fractions per row (sum=1 per group)
        group_ids: (N,) int32 tensor, group index per row
        eps:       small constant to avoid log(0)
    Returns:
        logits: (N,) float32 tensor
    """
    y_true = tf.reshape(y_true, [-1])
    group_ids = tf.reshape(group_ids, [-1])

    # clip to avoid log(0)
    y_true = tf.clip_by_value(y_true, eps, 1.0)

    # raw logits = log(p)
    raw_logits = tf.math.log(y_true)

    # subtract group mean so logits are centered (softmax is shift-invariant)
    num_groups = tf.reduce_max(group_ids) + 1
    group_means = tf.math.unsorted_segment_mean(raw_logits, group_ids, num_groups)
    centered_logits = raw_logits - tf.gather(group_means, group_ids)

    return centered_logits


def grouped_loss(y_true, logits, group_ids, ctrl_out, num_brands, ctrl_reg=1e-3, smooth=0.05, logit_reg=1e-4):
    """
    Grouped softmax loss with:
      - label smoothing to avoid 0/100 collapse
      - logit penalty to prevent extreme values
      - control penalty to keep controls small

    y_true:   (N,) float32, original KPI values (e.g. 0–100 scale)
    logits:   (N,) float32, model outputs before softmax
    group_ids:(N,) int32, group index for each row
    ctrl_out: (N,1) float32, control contribution
    num_brands: int, number of brands per group
    """

    y_true = tf.reshape(y_true, [-1])
    logits = tf.reshape(logits, [-1])
    group_ids = tf.reshape(group_ids, [-1])

    num_groups = tf.reduce_max(group_ids) + 1
    eps = 0

    # normalize y_true within each group → fractions
    group_sum_true = tf.math.unsorted_segment_sum(y_true, group_ids, num_groups)
    true_frac = y_true / (tf.gather(group_sum_true, group_ids) + eps)

    # # label smoothing
    # true_frac = true_frac * (1.0 - smooth) + smooth / float(num_brands)

    # predicted fractions (per-group softmax)
    logits = tf.clip_by_value(logits, -20.0, 20.0)  # stability
    exp_logits = tf.exp(logits - tf.reduce_max(logits))
    seg_sum = tf.math.unsorted_segment_sum(exp_logits, group_ids, num_groups)
    pred_frac = exp_logits / (tf.gather(seg_sum, group_ids) + eps)

    # mean squared error on fractions
    mse = tf.reduce_mean(tf.square(pred_frac - true_frac))
    # mse = tf.reduce_mean(tf.square(logits - fractions_to_logits_tf(y_true, group_ids)))

    # penalties
    ctrl_pen = tf.reduce_mean(tf.square(ctrl_out)) * ctrl_reg
    logit_pen = tf.reduce_mean(tf.square(logits)) * logit_reg

    return 100 * mse

In [12]:
# train
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

@tf.function
def train_step(batch_x, y_true, group_ids):
    with tf.GradientTape() as tape:
        logits, business_ctrl_out, global_trend, brand_trend, country_trend, season_out, time_float, baseline, variability = model(batch_x, training=True)
        # loss_value = grouped_loss_old(y_true, logits, group_ids, ctrl_out, ctrl_reg=1e-3)
        # loss_value = simple_loss(y_true, logits, ctrl_out, ctrl_reg=1e-1)
        # loss_value = mse_loss(y_true, logits)
        loss_value = grouped_loss(
            y_true=y_true,
            logits=logits,
            group_ids=group_ids,
            ctrl_out=business_ctrl_out,
            num_brands=5,
            ctrl_reg=1e-3,
            smooth=0.001,
            logit_reg=1e-5
        )
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value


for epoch in range(1000):
    epoch_losses = []
    for (batch_x, (y_true, group_ids)) in train_ds:  # your tf.data must yield this structure
        loss_value = train_step(batch_x, y_true, group_ids)
        epoch_losses.append(loss_value.numpy())
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: loss = {np.mean(epoch_losses):.4f}", flush=True)

Epoch 0: loss = 2.9595
Epoch 100: loss = 0.0100
Epoch 200: loss = 0.0032
Epoch 300: loss = 0.0019
Epoch 400: loss = 0.0023
Epoch 500: loss = 0.0015
Epoch 600: loss = 0.0015
Epoch 700: loss = 0.0016
Epoch 800: loss = 0.0010
Epoch 900: loss = 0.0014


In [13]:
# predict
logits, ctrl_out, global_trend, brand_trend, country_trend, season_out, time_out, baseline, variability = model(predict_x, training=False)
logits = logits.numpy().reshape(-1)
print(logits)

[-0.92709666 -0.8315175  -0.79436916 -0.8134334  -0.278816   -0.21156709
 -0.19948827 -0.24250686  0.2150197   0.28084773  0.28782466  0.23534474
 -0.63324064 -0.5509323  -0.52537584 -0.55635583  1.1144658   1.1814351
  1.1812061   1.1331245   2.6080806   2.6679132   2.6739576   2.6280894
 -1.6032448  -1.4899244  -1.4313824  -1.4311175  -0.3052748  -0.24065772
 -0.23037727 -0.2646      1.1296501   1.1936871   1.2155985   1.1600308
  0.37728253  0.35508755  0.331086    0.3529297   0.03809448  0.0104875
 -0.07363667  0.01114147 -0.64427847 -0.6622365  -0.6569074  -0.61562496
  1.5470072   1.5328352   1.5326341   1.5503167   1.6883523   1.6655006
  1.6325674   1.7075105   2.0982778   2.028952    1.999885    2.0488923
  2.2630944   2.2375689   2.2171755   2.2561836   0.4482983   0.43657723
  0.4043924   0.45973244  1.5606776   1.4994755   1.4906003   1.482777
  1.771851    1.7573508   1.7339902   1.7816205   1.771892    1.7778614
  1.7485867   1.7539607   0.25222892  0.27758166  0.34280357

In [14]:
print(baseline.numpy().reshape(-1))

[-0.9183292  -0.8243034  -0.7862264  -0.8056701  -0.26551098 -0.1973384
 -0.18577993 -0.22795838  0.21884678  0.2839143   0.29102215  0.24055624
 -0.63570106 -0.55209243 -0.52607083 -0.5566      1.1383585   1.2057576
  1.2036619   1.1526814   2.6102834   2.6699677   2.6765487   2.6303773
 -1.604387   -1.4866308  -1.431858   -1.4328893  -0.3042513  -0.23663765
 -0.22688872 -0.26488894  1.1468637   1.2148981   1.2218953   1.1737857
  0.3890115   0.35666844  0.33124146  0.36310136  0.04480943  0.01340157
 -0.01714462  0.01469985 -0.63764906 -0.65099317 -0.658245   -0.60785955
  1.5723767   1.5515202   1.5422409   1.5785365   1.6853428   1.6503837
  1.6185508   1.6431229   2.1148596   2.0900545   2.051413    2.0789835
  2.2657244   2.2403398   2.2201774   2.2584448   0.44705802  0.43378335
  0.4304558   0.46692336  1.5890453   1.5415196   1.4963906   1.5153558
  1.784617    1.7608702   1.7441845   1.7812028   1.7816699   1.7520373
  1.7332287   1.7725863   0.25392005  0.27880135  0.3447854

In [15]:
print(variability.numpy().reshape(-1))

[-8.76751263e-03 -7.21413223e-03 -8.14279448e-03 -7.76332850e-03
 -1.33050522e-02 -1.42286904e-02 -1.37083335e-02 -1.45484861e-02
 -3.82707966e-03 -3.06658447e-03 -3.19747906e-03 -5.21150744e-03
  2.46044109e-03  1.16014050e-03  6.94999821e-04  2.44165189e-04
 -2.38926988e-02 -2.43224986e-02 -2.24557817e-02 -1.95568949e-02
 -2.20270175e-03 -2.05451623e-03 -2.59122066e-03 -2.28776736e-03
  1.14228798e-03 -3.29364324e-03  4.75550420e-04  1.77184434e-03
 -1.02349708e-03 -4.02006833e-03 -3.48855229e-03  2.88945041e-04
 -1.72136053e-02 -2.12110300e-02 -6.29685633e-03 -1.37547888e-02
 -1.17289796e-02 -1.58088433e-03 -1.55459507e-04 -1.01716649e-02
 -6.71494892e-03 -2.91406782e-03 -5.64920530e-02 -3.55837913e-03
 -6.62938086e-03 -1.12433638e-02  1.33765617e-03 -7.76542583e-03
 -2.53695436e-02 -1.86850056e-02 -9.60671343e-03 -2.82198284e-02
  3.00953304e-03  1.51169822e-02  1.40166767e-02  6.43875152e-02
 -1.65817086e-02 -6.11025430e-02 -5.15280440e-02 -3.00913341e-02
 -2.62988638e-03 -2.77086

In [18]:
def logits_to_percentages(logits, group_ids, temp = 0.4):
    """
    Convert logits -> per-group percentages (summing to 100).
    
    logits:    (N,) array of raw model outputs
    group_ids: (N,) array of group IDs (int)
    """
    logits = np.asarray(logits).reshape(-1)
    group_ids = np.asarray(group_ids).reshape(-1)

    softmax = np.zeros_like(logits, dtype=float)

    for gid in np.unique(group_ids):
        mask = (group_ids == gid)
        group_logits = logits[mask]

        # stable softmax within group
        exp_logits = np.exp(group_logits - np.max(group_logits))
        softmax[mask] = exp_logits / exp_logits.sum()



    return (softmax) * 100 # percentages, rounded to 1 decimal


pred_pct = logits_to_percentages(logits, test_dict["group_id"])
# pred_pct = logits_to_percentages(logits, data_dict["output_data"]["group_id"])

In [19]:
# model predicted powers
# print(pred_pct)  
print(pred_pct.round(1))  

[ 1.7  1.7  1.8  1.8  3.2  3.2  3.2  3.2  5.3  5.3  5.3  5.2  2.3  2.3
  2.3  2.4 12.9 13.  12.8 12.8 57.6 57.4 57.2 57.2  0.9  0.9  0.9  1.
  3.1  3.1  3.1  3.2 13.1 13.1 13.3 13.2  3.   3.   3.   3.   2.1  2.1
  2.   2.1  1.1  1.1  1.1  1.1  9.6  9.7 10.   9.8 11.  11.1 11.  11.4
 16.6 16.  15.9 16.1 19.6 19.7 19.7 19.8  3.2  3.3  3.2  3.3  9.7  9.4
  9.5  9.1 12.  12.2 12.2 12.3 12.  12.4 12.4 12.   7.3  7.3  7.2  7.1
  3.1  3.1  3.1  3.1  1.2  1.2  1.2  1.2  1.6  1.6  1.6  1.6  3.2  3.2
  3.2  3.2  1.7  1.7  1.7  1.7  4.4  4.4  4.4  4.4 74.7 74.8 75.  75.1
  1.9  1.9  1.9  1.9  0.7  0.7  0.7  0.7]


In [22]:
# true powers
y_pred_true = test_dict["y_true"]
print(y_pred_true.round(1))

[ 2.8  2.9  3.1  3.2  3.3  3.3  3.4  3.6  5.   4.9  5.1  5.3  3.1  3.1
  3.4  3.5 11.4 11.9 11.4 11.  56.3 55.8 54.9 54.   2.   2.3  2.4  2.4
  3.3  3.2  3.1  3.1 12.8 12.7 13.2 13.8  3.2  3.3  3.1  2.9  2.   2.2
  2.1  2.2  1.6  1.7  1.8  1.9 10.  10.  10.  10.  10.4 10.3 10.4 10.6
 17.2 17.  16.1 16.  23.1 24.6 24.7 25.6  4.3  4.3  4.2  4.3  8.   7.6
  8.   7.4 15.2 14.4 15.  15.  15.1 14.6 14.6 14.1  5.8  5.8  6.1  5.8
  3.1  3.1  3.2  2.9  1.   1.1  1.1  1.1  1.6  1.7  1.7  1.8  3.2  2.9
  3.   2.9  1.4  1.5  1.3  1.5  4.   4.3  4.2  4.4 77.6 77.3 77.  77.
  1.6  1.7  1.5  1.7  0.7  0.7  0.8  0.8]


In [ ]:
RMSE = np.sqrt(np.mean((pred_pct[:1] - y_pred_true[:1])**2))
print(f"RMSE: {RMSE:.4f}")

# Output

In [ ]:
df_reference = test_dict["df_reference"].copy()
df_reference['predicted_power'] = pred_pct
df_reference['quarter'] = df_reference['quarter'].apply(lambda x: 'Qtr' + str(x))

df_reference

In [ ]:
normalize_c2 = df_reference.groupby(['country', 'year', 'quarter']).agg({'power': 'sum'}).reset_index()
normalize_c2 = normalize_c2[normalize_c2['country'] == 'a19bbfeafb']['power'].mean()
print(normalize_c2)

In [ ]:
df_reference['power'] = np.where(df_reference['country'] == 'a19bbfeafb', df_reference['power'] / normalize_c2 * 100, df_reference['power'])
df_reference.groupby(['country', 'year', 'quarter']).agg({'power': 'sum'}).reset_index()

In [ ]:
# df_reference.to_csv('../created_data/output_data_full_with_predictions.csv', index=False)
# preprocessed_data = pd.read_csv('../created_data/preprocessed_data_full.csv')
# preprocessed_data['quarter'] = preprocessed_data['quarter'].apply(lambda x: 'Qtr' + str(x))
# pd.concat([preprocessed_data, df_reference]).to_csv('../created_data/eda.csv', index=False)

In [ ]:
format_path = r'C:\Users\40107904\OneDrive - Anheuser-Busch InBev\ABI\WORK\hackathon_power\hackathon_lt_equity\data\lte_participants_data\BG_Data_Hackathon_Test_Predict_masked.csv'
format = pd.read_csv(format_path)
format

In [ ]:
format_path = r'C:\Users\40107904\OneDrive - Anheuser-Busch InBev\ABI\WORK\hackathon_power\hackathon_lt_equity\data\lte_participants_data\BG_Data_Hackathon_Test_Predict_masked.csv'
format = pd.read_csv(format_path)
format = format.merge(df_reference[['country', 'brand', 'year', 'quarter','power']], left_on=['Country', 'Brand', 'Year', 'Quarter'], right_on=['country', 'brand', 'year', 'quarter'], how='left')
format

In [ ]:
# format['Predicted Power'] = pred_pct
format['Predicted Power'] = format['power']
format.drop(columns=['country', 'brand', 'year', 'quarter', 'power'], inplace=True)
# format['Predicted Power'] = format['Predicted Power'].fillna(format['Predicted Power'].mean())
# format["Predicted Power"] = format["Predicted Power"].round(1)
format

In [ ]:
# format['Predicted Power'] = test_dict['y_true']
format

In [ ]:
format.to_csv('../submissions/comp_group_submission.csv', index=False)